In [1]:
import numpy as np
import pandas as pd
import scipy.stats

In [2]:
me_m = pd.read_csv('data\Portfolios_Formed_on_ME_monthly_EW.csv',
                    header=0, index_col=0, na_values=-99.99)
rets = me_m[['Lo 20', 'Hi 20']]
rets = rets/100
rets.index = pd.to_datetime(rets.index, format="%Y%m").to_period('M')

rets.head()

,Lo 20,Hi 20
1926-07,-0.0057,0.0333
1926-08,0.0384,0.0233
1926-09,-0.0048,-0.0009
1926-10,-0.0329,-0.0295
1926-11,-0.0055,0.0316


In [3]:
# Calculating annualized return
n = rets.shape[0]
ann_rets = np.power((1 + rets).prod(), 12/n) - 1
ann_rets * 100

Lo 20    15.197701
Hi 20     9.849021
dtype: float64

In [4]:
# Calculating annualized volatility
ann_vol = rets.std() * np.sqrt(12)
ann_vol * 100

Lo 20    33.670128
Hi 20    19.511604
dtype: float64

In [5]:
# Calculating annualized return between 1999 to 2015
n = rets['1999':'2015'].shape[0]
ann_rets = np.power((1 + rets['1999':'2015']).prod(), 12/n) - 1
ann_rets * 100

Lo 20    11.442634
Hi 20     6.286512
dtype: float64

In [6]:
# Calculating annualized volatility between 1999 to 2015
ann_vol = rets['1999':'2015'].std() * np.sqrt(12)
ann_vol * 100

Lo 20    22.889447
Hi 20    17.271332
dtype: float64

In [7]:
# Calculating maximum drawdown of Lo 20
wealth_index = 1000*(1 + rets['1999':'2015']['Lo 20']).cumprod()
previous_peaks = wealth_index.cummax()
drawdown = (wealth_index - previous_peaks)/previous_peaks

print('Max DD of Lo 20: ', -drawdown.min()*100)
print(drawdown.idxmin())

Max DD of Lo 20:  62.48092431236988
2009-02


In [8]:
# Calculating maximum drawdown of Hi 20
wealth_index = 1000*(1 + rets['1999':'2015']['Hi 20']).cumprod()
previous_peaks = wealth_index.cummax()
drawdown = (wealth_index - previous_peaks)/previous_peaks

print('Max DD of Hi 20: ', -drawdown.min()*100)
print(drawdown.idxmin())

Max DD of Hi 20:  55.27349978713653
2009-02


In [9]:
hfi = pd.read_csv('data\edhec-hedgefundindices.csv',
                  header=0, index_col=0, parse_dates=True, infer_datetime_format=True)
hfi = hfi/100
hfi.index = hfi.index.to_period('M')

hfi.head()

,Convertible Arbitrage,CTA Global,Distressed Securities,Emerging Markets,Equity Market Neutral,Event Driven,Fixed Income Arbitrage,Global Macro,Long/Short Equity,Merger Arbitrage,Relative Value,Short Selling,Funds Of Funds
date,,,,,,,,,,,,,
1997-01,0.0119,0.0393,0.0178,0.0791,0.0189,0.0213,0.0191,0.0573,0.0281,0.0150,0.0180,-0.0166,0.0317
1997-02,0.0123,0.0298,0.0122,0.0525,0.0101,0.0084,0.0122,0.0175,-0.0006,0.0034,0.0118,0.0426,0.0106
1997-03,0.0078,-0.0021,-0.0012,-0.0120,0.0016,-0.0023,0.0109,-0.0119,-0.0084,0.0060,0.0010,0.0778,-0.0077
1997-04,0.0086,-0.0170,0.0030,0.0119,0.0119,-0.0005,0.0130,0.0172,0.0084,-0.0001,0.0122,-0.0129,0.0009
1997-05,0.0156,-0.0015,0.0233,0.0315,0.0189,0.0346,0.0118,0.0108,0.0394,0.0197,0.0173,-0.0737,0.0275


In [10]:
hfi_A = hfi['2009':'2018']

# Checkinng the highest and lowest semideviation
hfi_A[hfi_A<0].std(ddof=0).sort_values()

Fixed Income Arbitrage    0.003321
Merger Arbitrage          0.003611
Global Macro              0.005075
Equity Market Neutral     0.005186
Relative Value            0.005711
Convertible Arbitrage     0.006681
Funds Of Funds            0.007948
Distressed Securities     0.009498
Event Driven              0.010135
CTA Global                0.010996
Long/Short Equity         0.012139
Emerging Markets          0.015276
Short Selling             0.021240
dtype: float64

In [11]:
skw = pd.DataFrame(scipy.stats.skew(hfi_A))
col = pd.DataFrame(hfi_A.columns.values)

skewness = pd.concat([col, skw], axis=1)
skewness.columns = ['Hedge Fund Indices', 'Skewness']

skewness

,Hedge Fund Indices,Skewness
0,Convertible Arbitrage,1.305911
1,CTA Global,0.052062
2,Distressed Securities,-0.254944
3,Emerging Markets,0.033123
4,Equity Market Neutral,-0.896327
5,Event Driven,-0.488821
6,Fixed Income Arbitrage,1.121453
7,Global Macro,0.348184
8,Long/Short Equity,-0.463703
9,Merger Arbitrage,-0.551065


In [12]:
# Getting the hedge fund indices with the most negatively skewed

skewness['Skewness'].sort_values()

4    -0.896327
12   -0.646908
9    -0.551065
5    -0.488821
8    -0.463703
2    -0.254944
3     0.033123
1     0.052062
10    0.159953
7     0.348184
11    0.456518
6     1.121453
0     1.305911
Name: Skewness, dtype: float64

In [13]:
hfi_B = hfi['2000':]

krts = pd.DataFrame(scipy.stats.kurtosis(hfi_B) + 3)
col = pd.DataFrame(hfi_B.columns.values)

kurtosis = pd.concat([col, krts], axis=1)
kurtosis.columns = ['Hedge Fund Indices', 'Kurtosis']

kurtosis

,Hedge Fund Indices,Kurtosis
0,Convertible Arbitrage,23.001449
1,CTA Global,2.845022
2,Distressed Securities,6.231720
3,Emerging Markets,5.941289
4,Equity Market Neutral,20.329621
5,Event Driven,5.543444
6,Fixed Income Arbitrage,33.378729
7,Global Macro,3.653278
8,Long/Short Equity,4.472589
9,Merger Arbitrage,4.520785


In [14]:
# Getting the hedge fund index with highest kurtosis
kurtosis['Kurtosis'].sort_values()

1      2.845022
7      3.653278
11     4.352758
8      4.472589
9      4.520785
5      5.543444
3      5.941289
2      6.231720
12     7.584725
10    12.731096
4     20.329621
0     23.001449
6     33.378729
Name: Kurtosis, dtype: float64